In [ ]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
%matplotlib inline 

In [ ]:
from mrcnn.config import Config
from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
import mrcnn.model as modellib
from mrcnn.model import log

In [ ]:
## Kaggle에서 2018 Data science bowl Nucleus segmentation 데이터를 download 한 뒤 ./nucleus_data 디렉토리에 압축을 품
# stage1_train.zip 파일에 train용 image 데이터와 mask 데이터가 있음. stage1_train_zip 파일을 stage1_train 디렉토리에 압축 해제
# unzip stage1_train.zip -d stage1_train
# stage1_test.zip 파일에 test용 image 데이터와 mask 데이터가 있음. stage1_test_zip 파일을 stage1_test 디렉토리에 압축 해제
# unzip stage1_test.zip -d stage1_test

**train용 데이터 세트에 들어있는 데이터 구조 확인**

In [ ]:
import os
from pathlib import Path

HOME_DIR = str(Path.home())

# 학습용, 테스트용 모두의 기준 디렉토리는 ~/DLCV/data/nucleus 임. 
DATASET_DIR = os.path.join(HOME_DIR, "DLCV/data/nucleus")
print(DATASET_DIR)

In [ ]:
# ~/DLCV/data/nucleus 디렉토리 밑에 학습용 디렉토리인 stage1_train이 만들어짐.stage1_train에 학습 이미지, mask 이미지 데이터 존재. 
subset_dir = 'stage1_train'
train_dataset_dir = os.path.join(DATASET_DIR, subset_dir)
print(train_dataset_dir)

In [ ]:
# train 데이터 세트의 이미지 파일, mask 파일이 어떠한 디렉토리 구조 형태로 저장되어 있는지 확인. 

In [ ]:
# 이미지 별로 고유한 이미지명을 가지는 이미지 디렉토리를 가지고 이 디렉토리에 하위 디렉토리로 images, masks를 가짐
# images 에는 하나의 이미지가 있으며 masks는 여러개의 mask 이미지 파일을 가지고 있음. 즉 하나의 이미지에 여러개의 mask 파일을 가지고 있는 형태임. 
# next(os.walk(directory))[1]은 sub directory를 iteration으로 반환 next(os.walk(directory))[2]는 해당 디렉토리 밑에 파일들을 iteration으로 반환
index = 0 
for dir in next(os.walk(train_dataset_dir))[1]:
    print('',dir)
    subdirs = os.path.join(train_dataset_dir, dir)
    for subdir in next(os.walk(subdirs))[1]:
        print('----'+subdir)
        sub_subdirs = os.path.join(subdirs, subdir)
        for sub_subdir in next(os.walk(sub_subdirs))[2]:
            print('    ---- '+sub_subdir)
            index += 1
            if index >1000:
                break

In [ ]:
### 학습시 사용할 임의의 Validation용 IMAGE ID를 별도로 선정. 

In [ ]:
np.random.seed(0)
def get_valid_image_ids(dataset_dir, valid_size):
    
    dataset_dir = os.path.join(dataset_dir,'stage1_train')
    image_ids = next(os.walk(dataset_dir))[1]
    total_cnt = len(image_ids)
    
    valid_cnt = int(total_cnt * valid_size)
    valid_indexes = np.random.choice(total_cnt, valid_cnt, replace=False)
    
    return list(np.array(image_ids)[valid_indexes])

In [ ]:
get_valid_image_ids(DATASET_DIR, 0.1)

In [ ]:
## Dataset객체의 add_image()를 이용하여 개별 이미지를 Dataset 객체로 로딩하는 load_nucleus() 함수 생성. 

In [ ]:
from mrcnn import utils
import skimage
import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

VAL_IMAGE_IDS = get_valid_image_ids(DATASET_DIR, 0.1)

class NucleusDataset(utils.Dataset):
    
    # subset은 train, valid, stage1_test, stage2_test
    def load_nucleus(self, dataset_dir, subset):
        self.add_class(source='nucleus', class_id=1, class_name='nucleus')
        
        subset_dir = 'stage1_train' if subset in ['train', 'val'] else subset
        dataset_dir = os.path.join(dataset_dir, subset_dir)
        
        if subset=='val':
            image_ids = VAL_IMAGE_IDS
        else:
            image_ids = next(os.walk(dataset_dir))[1]
            if subset=='train':
                image_ids = list(set(image_ids) - set(VAL_IMAGE_IDS))

        for image_id in image_ids:       
            self.add_image('nucleus', image_id=image_id, path=os.path.join(dataset_dir, image_id, 'images/{}.png'.format(image_id)))
    
    def load_mask(self, image_id):
        info = self.image_info[image_id]
        mask_dir = os.path.join(os.path.dirname(os.path.dirname(info['path'])), 'masks')
        mask_list=[]
        for mask_file in next(os.walk(mask_dir))[2]:
            if mask_file.endswith('.png'):
                mask = skimage.io.imread(os.path.join(mask_dir, mask_file)).astype(np.bool)
                mask_list.append(mask)
                
        masks = np.stack(mask_list, axis=-1)
        
        return masks, np.ones([masks.shape[-1]], dtype=np.int32)

In [ ]:
nucleus_dataset = NucleusDataset(utils.Dataset)
nucleus_dataset.load_nucleus(DATASET_DIR, 'train')
#print('class info:', nucleus_dataset.class_info)
#print('image info:', nucleus_dataset.image_info)
nucleus_dataset.prepare()

In [ ]:
print('class id:', nucleus_dataset.class_ids)
print('image id:', nucleus_dataset._image_ids)

In [ ]:
nucleus_dataset.image_info[0]

In [ ]:
masks, class_ids = nucleus_dataset.load_mask(0)
masks.shape, class_ids.shape

In [ ]:
masks, class_ids = nucleus_dataset.load_mask(0)

sample_img = skimage.io.imread(os.path.join(DATASET_DIR, 'stage1_train', 
                        'df53d0b6c2c4e45d759b2c474011e2b2b32552cd100ca4b22388ab9ca1750ee2',
                        'images',
                        'df53d0b6c2c4e45d759b2c474011e2b2b32552cd100ca4b22388ab9ca1750ee2.png'))
print('image shape:', sample_img.shape)
plt.imshow(sample_img)
plt.show()

print(masks[0])

In [ ]:
np.random.seed(0)
image_ids = np.random.choice(nucleus_dataset.image_ids, 4)
print(image_ids)
for image_id in image_ids:
    image = nucleus_dataset.load_image(image_id)
    mask, class_ids = nucleus_dataset.load_mask(image_id)
    print('mask shape:', mask.shape, 'class_ids shape:', class_ids.shape)
    visualize.display_top_masks(image, mask, class_ids, nucleus_dataset.class_names, limit=1)

### nucleus 데이터 세트 Train/Inference 
* Matterport 패키지에서 사용될 수 있도록 학습/검증/테스트 데이터 세트를 변환하여 생성 필요
* 학습 또는 Inference를 위한 Config 설정.  

In [ ]:
dataset_train = NucleusDataset()
dataset_train.load_nucleus(DATASET_DIR, 'train')
# dataset을 load한 뒤에는 반드시 prepare()메소드를 호출
dataset_train.prepare()

# Validation dataset
dataset_val = NucleusDataset()
dataset_val.load_nucleus(DATASET_DIR, "val")
dataset_val.prepare()

In [ ]:
len(dataset_train.image_info), len(dataset_val.image_info)

#### Nucleus 학습을 위한 새로운 Config 생성

In [ ]:
from mrcnn.config import Config

train_image_cnt = len(dataset_train.image_info)
val_image_cnt = len(dataset_val.image_info)
print('train_image_cnt:',train_image_cnt, 'validation image count:',val_image_cnt)

class NucleusConfig(Config):
    """Configuration for training on the nucleus segmentation dataset."""
    # Give the configuration a recognizable name
    NAME = "nucleus"

    # Adjust depending on your GPU memory
    IMAGES_PER_GPU = 6

    # Number of classes (including background)
    NUM_CLASSES = 1 + 1  # Background + nucleus

    # Number of training and validation steps per epoch
    STEPS_PER_EPOCH = (train_image_cnt) // IMAGES_PER_GPU
    VALIDATION_STEPS = max(1, (val_image_cnt // IMAGES_PER_GPU))

    # Don't exclude based on confidence. Since we have two classes
    # then 0.5 is the minimum anyway as it picks between nucleus and BG
    DETECTION_MIN_CONFIDENCE = 0

    # Backbone network architecture
    # Supported values are: resnet50, resnet101
    BACKBONE = "resnet50"

    # Input image resizing
    # Random crops of size 512x512
    IMAGE_RESIZE_MODE = "crop"
    IMAGE_MIN_DIM = 512
    IMAGE_MAX_DIM = 512
    IMAGE_MIN_SCALE = 2.0

    # Length of square anchor side in pixels
    RPN_ANCHOR_SCALES = (8, 16, 32, 64, 128)

    # ROIs kept after non-maximum supression (training and inference)
    POST_NMS_ROIS_TRAINING = 1000
    POST_NMS_ROIS_INFERENCE = 2000

    # Non-max suppression threshold to filter RPN proposals.
    # You can increase this during training to generate more propsals.
    RPN_NMS_THRESHOLD = 0.9

    # How many anchors per image to use for RPN training
    RPN_TRAIN_ANCHORS_PER_IMAGE = 64

    # Image mean (RGB)
    MEAN_PIXEL = np.array([43.53, 39.56, 48.22])

    # If enabled, resizes instance masks to a smaller size to reduce
    # memory load. Recommended when using high-resolution images.
    USE_MINI_MASK = True
    MINI_MASK_SHAPE = (56, 56)  # (height, width) of the mini-mask

    # Number of ROIs per image to feed to classifier/mask heads
    # The Mask RCNN paper uses 512 but often the RPN doesn't generate
    # enough positive proposals to fill this and keep a positive:negative
    # ratio of 1:3. You can increase the number of proposals by adjusting
    # the RPN NMS threshold.
    TRAIN_ROIS_PER_IMAGE = 128

    # Maximum number of ground truth instances to use in one image
    MAX_GT_INSTANCES = 200

    # Max number of final detections per image
    DETECTION_MAX_INSTANCES = 400

#### coco  pretrained된 가중치 모델 위치 지정. 

In [ ]:
ROOT_DIR = os.path.abspath("./")
# Path to trained weights file
COCO_WEIGHTS_PATH = os.path.join(ROOT_DIR, "./pretrained/mask_rcnn_coco.h5")

# Directory to save logs and model checkpoints, if not provided
# through the command line argument --logs
MODEL_DIR = os.path.join(ROOT_DIR, "./snapshots/nucleus")

#### 학습을 위한 모델을 생성, Mask_RCNN 패키지는 modellib에 MaskRCNN 객체를 이용하여 Mask RCNN 모델을 생성함. 

생성 인자
* mode: training인지 inference인지 설정
* config: training 또는 inference에 따라 다른 config 객체 사용. Config객체를 상속 받아 각 경우에 새로운 객체를 만들고 이를 이용. 
inference 시에는 Image를 하나씩 입력 받아야 하므로 Batch size를 1로 만들 수 있도록 설정 
            
* model_dir: 학습 진행 중에 Weight 모델이 저장되는 장소 지정.   

In [ ]:
from mrcnn import model as modellib

train_config = NucleusConfig()
train_config.display()

model = modellib.MaskRCNN(mode="training", config=train_config,
                                  model_dir=MODEL_DIR)

# Exclude the last layers because they require a matching
# number of classes
model.load_weights(COCO_WEIGHTS_PATH, by_name=True, exclude=["mrcnn_class_logits", "mrcnn_bbox_fc","mrcnn_bbox", "mrcnn_mask"])

#### 학습 데이터 세트와 검증 데이터 세트를 NucleusDataset 객체에 로드하고 train 시작
* augmentation은 imgaug를 사용. 

In [ ]:
from imgaug import augmenters as iaa

img_aug = iaa.SomeOf((0, 2), [
        iaa.Fliplr(0.5),
        iaa.Flipud(0.5),
        iaa.OneOf([iaa.Affine(rotate=90),
                   iaa.Affine(rotate=180),
                   iaa.Affine(rotate=270)]),
        iaa.Multiply((0.8, 1.5)),
        iaa.GaussianBlur(sigma=(0.0, 5.0))
    ])

In [ ]:
import warnings
warnings.filterwarnings('ignore')

print("Train all layers")
model.train(dataset_train, dataset_val,
            learning_rate=train_config.LEARNING_RATE,
            epochs=40, augmentation=img_aug,
            layers='all')

#### 학습 모델을 이용하여 Inference 수행

In [ ]:
## 예측용 모델을 로드. mode는 inference로 설정,config는 NucleusInferenceConfig()로 설정,
## 예측용 모델에 위에서 찾은 학습 중 마지막 저장된 weight파일을 로딩함. 

In [ ]:
class NucleusInferenceConfig(NucleusConfig):
    NAME='nucleus'
    # 이미지 한개씩 차례로 inference하므로 batch size를 1로 해야 하며 이를 위해 IMAGES_PER_GPU = 1
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    # pad64는 64 scale로 이미지를 맞춰서 resize수행. 
    IMAGE_RESIZE_MODE = "pad64"
    # Non-max suppression threshold to filter RPN proposals.
    # You can increase this during training to generate more propsals.
    RPN_NMS_THRESHOLD = 0.7


In [ ]:
infer_config = NucleusInferenceConfig()
inference_model = modellib.MaskRCNN(mode="inference", config=infer_config, model_dir=MODEL_DIR)
weights_path = model.find_last()
print('학습중 마지막으로 저장된 weight 파일:', weights_path)
inference_model.load_weights(weights_path, by_name=True)


In [ ]:
### 테스트용 데이터 세트를 NucleusDataset으로 로딩. load_nucleus() 테스트 세트를 지정하는 'stage1_test'를 입력. 
dataset_test = NucleusDataset()
dataset_test.load_nucleus(DATASET_DIR, 'stage1_test')
dataset_test.prepare()

In [ ]:
dataset_test.image_ids

In [ ]:
### mask detected 된 임의의 파일을 시각화

In [ ]:
for image_id in dataset_test.image_ids:
        # Load image and run detection
        image = dataset_test.load_image(image_id)
        print(len(image))
        # Detect objects
        r = inference_model.detect([image], verbose=0)[0]
        # Save image with masks
        visualize.display_instances(
            image, r['rois'], r['masks'], r['class_ids'],
            dataset_test.class_names, r['scores'],
            show_bbox=False, show_mask=False,
            title="Predictions")

In [ ]:
for image_id in dataset_test.image_ids:
        # Load image and run detection
        image = dataset_test.load_image(image_id)
        print(len(image))
       
        #plt.savefig("{}/{}.png".format(submit_dir, dataset.image_info[image_id]["id"]))

In [ ]:
detect(model, args.dataset, args.subset)

In [ ]:
def detect(model, dataset_dir, subset):
    """Run detection on images in the given directory."""
    print("Running on {}".format(dataset_dir))

    # Create directory
    if not os.path.exists(RESULTS_DIR):
        os.makedirs(RESULTS_DIR)
    submit_dir = "submit_{:%Y%m%dT%H%M%S}".format(datetime.datetime.now())
    submit_dir = os.path.join(RESULTS_DIR, submit_dir)
    os.makedirs(submit_dir)

    # Read dataset
    dataset = NucleusDataset()
    dataset.load_nucleus(dataset_dir, subset)
    dataset.prepare()
    # Load over images
    submission = []
    for image_id in dataset.image_ids:
        # Load image and run detection
        image = dataset.load_image(image_id)
        # Detect objects
        r = model.detect([image], verbose=0)[0]
        # Encode image to RLE. Returns a string of multiple lines
        source_id = dataset.image_info[image_id]["id"]
        rle = mask_to_rle(source_id, r["masks"], r["scores"])
        submission.append(rle)
        # Save image with masks
        visualize.display_instances(
            image, r['rois'], r['masks'], r['class_ids'],
            dataset.class_names, r['scores'],
            show_bbox=False, show_mask=False,
            title="Predictions")
        plt.savefig("{}/{}.png".format(submit_dir, dataset.image_info[image_id]["id"]))

    # Save to csv file
    submission = "ImageId,EncodedPixels\n" + "\n".join(submission)
    file_path = os.path.join(submit_dir, "submit.csv")
    with open(file_path, "w") as f:
        f.write(submission)
    print("Saved to ", submit_dir)